In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import time
from numpy import load
import torch
from sklearn.metrics import classification_report
from tqdm import tqdm
from sklearn.metrics import f1_score, roc_auc_score, average_precision_score, matthews_corrcoef, confusion_matrix
import keras
from keras import layers
import tensorflow as tf

2023-06-30 15:13:44.640788: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:7630] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-06-30 15:13:44.640829: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-06-30 15:13:44.640847: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-30 15:13:44.649051: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-30 15:13:45.513748: W tensorflow/compiler/

Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.


In [2]:
SESSIONS = ['05', '06', '07', '08', '09', '11', '12', '13', '14']
data = {}
for s in SESSIONS:
    data[s] =  np.load('data/markered_data/{0}/data_8x8x48_{0}.npz'.format(s, s))

In [3]:
X_train_pure_data = np.concatenate([data['05']['arr_0'], data['06']['arr_0'], data['07']['arr_0'], data['08']['arr_0'],
                    data['09']['arr_0'], data['11']['arr_0'], data['12']['arr_0'], data['13']['arr_0']], axis = 0)

Y_train_pure_data = np.concatenate([data['05']['arr_1'], data['06']['arr_1'], data['07']['arr_1'], data['08']['arr_1'],
                    data['09']['arr_1'], data['11']['arr_1'], data['12']['arr_1'], data['13']['arr_1']], axis = 0)

X_test_pure_data = data['14']['arr_0']

Y_test_pure_data = data['14']['arr_1']


X_train_pure_data = preprocessing.normalize(X_train_pure_data)
# Y_train_pure_data[Y_train_pure_data <= 0] = -1
X_test_pure_data = preprocessing.normalize(X_test_pure_data)
# Y_test_pure_data[Y_test_pure_data <= 0] = -1

### Логистическая регрессия

In [5]:
solvers = ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']
TRAIN_RESULTS = {}
TEST_RESULTS = {}


for slv in solvers:
    print(slv)
    #model creating
    model = LogisticRegression(solver=slv, max_iter=300, random_state=0, C=0.7).fit(X_train_pure_data, Y_train_pure_data)
    
    preds_proba_train = model.predict_proba(X_train_pure_data)[:, 1]
    preds_proba_test = model.predict_proba(X_test_pure_data)[:, 1]
    preds_train = model.predict(X_train_pure_data)
    preds_test = model.predict(X_test_pure_data)
    
    TRAIN_RESULTS[slv] = [roc_auc_score(Y_train_pure_data, preds_proba_train), 
                          average_precision_score(Y_train_pure_data, preds_proba_train), 
                          matthews_corrcoef(Y_train_pure_data, preds_train),
                        f1_score(Y_train_pure_data, preds_train)]
    TEST_RESULTS[slv] = [roc_auc_score(Y_test_pure_data, preds_proba_test), 
                          average_precision_score(Y_test_pure_data, preds_proba_test), 
                          matthews_corrcoef(Y_test_pure_data, preds_test),
                        f1_score(Y_test_pure_data, preds_test)]

lbfgs
liblinear
newton-cg
newton-cholesky
sag
saga


In [4]:
TEST_RESULTS, TRAIN_RESULTS

NameError: name 'TEST_RESULTS' is not defined

### Нейронная сеть

In [4]:
def build_model(params):
    l_r, loss_function, activation_fucntion, hidden_layer, hidden_layer_2, input_layer, output_layer, metr = params[0], params[1], params[2], params[3], params[4], params[5], params[6], params[7]
    print('l_r:', l_r, 'loss_function:',loss_function, 'activation_fucntion:', activation_fucntion,
          'hidden_layer:', hidden_layer, 'hidden_layer_2:', hidden_layer_2,
          'input_layer:', input_layer, 'output_layer:', output_layer, 'metr:', metr)
    model = keras.Sequential()
    model.add(
        layers.Dense(
            units = hidden_layer,
            activation="relu",
            input_shape=(input_layer,)
        )
    )
    model.add(
        layers.Dense(
            units = hidden_layer_2,
            activation="relu",
            input_shape=(hidden_layer,)
        )
    )

    model.add(
        layers.Dense(
            units = output_layer,
            activation = 'sigmoid')
        )
    #////////////////////////////////////////////////////////
    model.compile(
        optimizer = keras.optimizers.Adam(learning_rate = l_r),
        # optimizer = tf.keras.optimizers.legacy.SGD(learning_rate = l_r),
#         optimizer = tf.keras.optimizers.legacy.SGD(learning_rate = l_r, nesterov = True),
        loss = loss_function,
        metrics = metr
    )

    return model

In [5]:
roc_auc = tf.keras.metrics.AUC(
    num_thresholds=200,
    curve='ROC',
    summation_method='interpolation',
    name=None,
    dtype=None,
    thresholds=None,
    multi_label=False,
    num_labels=None,
    label_weights=None,
    from_logits=False
)

pr_auc = tf.keras.metrics.AUC(
    num_thresholds=200,
    curve='PR',
    summation_method='interpolation',
    name=None,
    dtype=None,
    thresholds=None,
    multi_label=False,
    num_labels=None,
    label_weights=None,
    from_logits=False
)

f1_score = tf.keras.metrics.F1Score(
    average='micro', threshold=0.5, name='f1_score', dtype=None
)

In [6]:
def validation(X_train, Y_train, X_test, Y_test, params, BATCHES, EPOCHS_COUNT):

    #создаётся модель с определёнными ранее параметрами
    model = build_model(params)

    train_history = model.fit(X_train, Y_train, batch_size = BATCHES, epochs = EPOCHS_COUNT)
    results_test = model.evaluate(X_test, Y_test, batch_size = BATCHES)
    
    y_train_pred_proba = model.predict(X_train)
    y_test_pred_proba = model.predict(X_test)
    y_train_pred = y_train_pred_proba.copy()
    y_test_pred = y_test_pred_proba.copy()
    
    y_train_pred[y_train_pred >= 0.5] = 1
    y_train_pred[y_train_pred < 0.5] = 0
    y_test_pred[y_test_pred >= 0.5] = 1
    y_test_pred[y_test_pred < 0.5] = 0
    
    train_score = [roc_auc(Y_train, y_train_pred_proba), 
                   pr_auc(Y_train, y_train_pred_proba),
                   matthews_corrcoef(Y_train, y_train_pred),
                   train_history.history['f1_score'][-1],
                   (confusion_matrix(Y_train, y_train_pred).ravel())
                  ]
    test_score = [roc_auc(Y_test, y_test_pred_proba), 
                  pr_auc(Y_test, y_test_pred_proba), 
                  matthews_corrcoef(Y_test, y_test_pred),
                  results_test[1],
                  (confusion_matrix(Y_test, y_test_pred).ravel())
                 ]
    
    return train_score, test_score

In [7]:
def fit(X_train, Y_train, X_test, Y_test, l_rates, activation, loss_function, input_layer, hidden_layers_1,
                  hidden_layers_2, output_layer, metr):
    RESULT = {}
    PROGONS = 1
    for progon in range(PROGONS):
        print("progon # =", progon)
        for activ in activation:
            for h_1 in hidden_layers_1:
                for h_2 in hidden_layers_2:
                    print('##################################################################################', '\n'
                          '| Число узлов первого и второго скрытых слоёв:', h_1, " ", h_2)
                    params = [l_rates, loss_function, activ, h_1, h_2, input_layer, output_layer, metr]
                    TRAIN_RESULTS, TEST_RESULTS = validation(X_train, Y_train, X_test, Y_test, params, BATCHES = 254, EPOCHS_COUNT = 150)
                    RESULT[' '.join(map(str, params))] = [TEST_RESULTS, TRAIN_RESULTS]

    return RESULT

In [41]:
initial_learning_rate = 0.005
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=100000,
    decay_rate=0.96,
    staircase=True)


res = fit(X_train_pure_data, Y_train_pure_data, X_test_pure_data, Y_test_pure_data, l_rates = lr_schedule, 
          #'relu', 'sigmoid', 'tanh'
          loss_function = keras.losses.BinaryCrossentropy(), activation = ['tanh'],
          hidden_layers_1 = [128], hidden_layers_2 = [32], input_layer = 48, output_layer = 1, metr = f1_score)

progon # = 0
################################################################################## 
| Число узлов первого и второго скрытых слоёв: 128   32
l_r: <keras.src.optimizers.schedules.learning_rate_schedule.ExponentialDecay object at 0x7fb8ded3e9a0> loss_function: <keras.src.losses.BinaryCrossentropy object at 0x7fb8ded3e0a0> activation_fucntion: tanh hidden_layer: 128 hidden_layer_2: 32 input_layer: 48 output_layer: 1 metr: F1Score(name=f1_score,dtype=float32,average=micro,threshold=0.5)
Epoch 1/150
2033/2033 [==============================] - 5s 2ms/step - loss: 0.2653 - f1_score: 0.1537
Epoch 2/150
2033/2033 [==============================] - 4s 2ms/step - loss: 0.2153 - f1_score: 0.3232
Epoch 3/150
2033/2033 [==============================] - 4s 2ms/step - loss: 0.1726 - f1_score: 0.5225
Epoch 4/150
2033/2033 [==============================] - 4s 2ms/step - loss: 0.1576 - f1_score: 0.5753
Epoch 5/150
2033/2033 [==============================] - 4s 2ms/step - loss: 0.1441 - f1

2033/2033 [==============================] - 4s 2ms/step - loss: 0.0914 - f1_score: 0.7764
Epoch 76/150
2033/2033 [==============================] - 4s 2ms/step - loss: 0.0922 - f1_score: 0.7752
Epoch 77/150
2033/2033 [==============================] - 4s 2ms/step - loss: 0.0907 - f1_score: 0.7791
Epoch 78/150
2033/2033 [==============================] - 4s 2ms/step - loss: 0.0915 - f1_score: 0.7767
Epoch 79/150
2033/2033 [==============================] - 4s 2ms/step - loss: 0.0917 - f1_score: 0.7769
Epoch 80/150
2033/2033 [==============================] - 4s 2ms/step - loss: 0.0914 - f1_score: 0.7777
Epoch 81/150
2033/2033 [==============================] - 4s 2ms/step - loss: 0.0905 - f1_score: 0.7795
Epoch 82/150
2033/2033 [==============================] - 4s 2ms/step - loss: 0.0914 - f1_score: 0.7791
Epoch 83/150
2033/2033 [==============================] - 4s 2ms/step - loss: 0.0904 - f1_score: 0.7796
Epoch 84/150
2033/2033 [==============================] - 4s 2ms/step - loss:

In [42]:
res

{'<keras.src.optimizers.schedules.learning_rate_schedule.ExponentialDecay object at 0x7fb8ded3e9a0> <keras.src.losses.BinaryCrossentropy object at 0x7fb8ded3e0a0> tanh 128 32 48 1 F1Score(name=f1_score,dtype=float32,average=micro,threshold=0.5)': [[<tf.Tensor: shape=(), dtype=float32, numpy=0.9727864>,
   <tf.Tensor: shape=(), dtype=float32, numpy=0.8697278>,
   0.8044419674120792,
   0.8132655620574951,
   array([60401,   393,  1223,  3519])],
  [<tf.Tensor: shape=(), dtype=float32, numpy=0.9727289>,
   <tf.Tensor: shape=(), dtype=float32, numpy=0.8696118>,
   0.7941929854923013,
   0.7960426807403564,
   array([470559,   6511,   8117,  31037])]]}

_________________________________________

In [57]:
EPOCHS = 50
LEARNING_RATE = 0.001

In [ ]:
## train data
from torch.utils.data import Dataset, DataLoader

class TrainData(Dataset):

    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]

    def __len__ (self):
        return len(self.X_data)

## test data
class TestData(Dataset):

    def __init__(self, X_data, y_data):
        self.X_data = X_data
        self.y_data = y_data

    def __getitem__(self, index):
        return self.X_data[index], self.y_data[index]

    def __len__ (self):
        return len(self.X_data)

In [ ]:
import torch.nn as nn
class BinaryClassification(nn.Module):
    def __init__(self):
        super(BinaryClassification, self).__init__()
        # Number of input features is 48.
        self.layer_1 = nn.Linear(48, 96)
        self.layer_2 = nn.Linear(96, 64)
        self.layer_out = nn.Linear(64, 1)

        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=0.1)
        self.batchnorm1 = nn.BatchNorm1d(96)
        self.batchnorm2 = nn.BatchNorm1d(64)

    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.batchnorm1(x)
        x = self.relu(self.layer_2(x))
        x = self.batchnorm2(x)
        x = self.dropout(x)
        x = self.layer_out(x)

        return x

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [ ]:
def binary_acc(y_pred, y_test):
    y_pred_tag = torch.round(torch.sigmoid(y_pred))

    correct_results_sum = (y_pred_tag == y_test).sum().float()
    acc = correct_results_sum/y_test.shape[0]
    acc = torch.round(acc * 100)

    return acc

In [ ]:
def create_train_test_data(test_s, data):
    train_X_data = []
    train_Y_data = []

    for s in SESSION:
        if s != test_s:
            train_X_data.append(data[s]['arr_0'])
            train_Y_data.append(data[s]['arr_1'])

    X_train = preprocessing.normalize(np.concatenate(train_X_data))
    Y_train = np.concatenate(train_Y_data)

    X_test = preprocessing.normalize(data[test_s]['arr_0'])
    Y_test = data[test_s]['arr_1']

    train_data = TrainData(torch.FloatTensor(X_train), torch.FloatTensor(Y_train))
    test_data = TestData(torch.FloatTensor(X_test), torch.FloatTensor(Y_test))

    return train_data, test_data

In [ ]:
from torchmetrics import AUROC, MatthewsCorrCoef, PrecisionRecallCurve, AveragePrecision
from sklearn.metrics import confusion_matrix

def metrics(model, data):
    X = data.X_data
    Y = data.y_data.int()
    X_loader = DataLoader(dataset=X, batch_size=len(X))

    y_pred_list = []
    model.eval()

    with torch.no_grad():
        for X_batch in X_loader:
            X_batch = X_batch.to(device)
            y_pred = torch.sigmoid(model(X_batch))
            y_pred_tag = torch.round(y_pred)
            y_pred_list.append(y_pred_tag.cpu().numpy())

    y_pred_list = [a.squeeze().tolist() for a in y_pred_list][0]

    tn, fp, fn, tp = confusion_matrix(Y, y_pred_list).ravel()

    auroc = AUROC(task="binary")
    average_precision = AveragePrecision(task="binary")
    matthews_corrcoef = MatthewsCorrCoef(task='binary')

    Y_pred_tensor = torch.tensor(y_pred_list)
    PR_AUC = average_precision(Y_pred_tensor, Y)
    ROC_AUC = auroc(Y_pred_tensor, Y)
    MCC = matthews_corrcoef(Y_pred_tensor, Y)
    print(ROC_AUC, PR_AUC, MCC)
    return ROC_AUC, PR_AUC, (tn, fp, fn, tp)

In [ ]:
def clf_fit(train_data, model, criterion, optimizer):
    model.train()
    losses = []
    for e in range(1, 8):
        epoch_loss = 0
        epoch_acc = 0
        train_loader = DataLoader(dataset = train_data, batch_size = 128, shuffle = True)
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            y_pred = model(X_batch)

            loss = criterion(y_pred, y_batch.unsqueeze(1))
            acc = binary_acc(y_pred, y_batch.unsqueeze(1))

            loss.backward()
            optimizer.step()

            epoch_loss += loss.item()
            epoch_acc += acc.item()

        losses.append(epoch_loss)

        print(f'Epoch {e+0:03}: | Loss: {epoch_loss/len(train_loader):.5f} | Acc: {epoch_acc/len(train_loader):.3f}')
    return losses, model

def cross_val_score(data, LEARNING_RATE = 0.001):
    TRAIN_RESULTS_BY_SESSION = {}
    TEST_RESULTS_BY_SESSION = {}
    LOSSES = {}

    for s in SESSION:
        print("SESSION №", s)
        #каждая сессия выбирается валидационной, остальные обучающими
        train_data, test_data = create_train_test_data(s, data)

        #создаётся модель с определёнными ранее параметрами
        model = BinaryClassification()
        model.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

        #обучим модель
        losses, trained_model = clf_fit(train_data, model, criterion, optimizer)
        LOSSES[s] = losses

        #метрика
        AUC_train, MCC_train, (tn_train, fp_train, fn_train, tp_train) = metrics(trained_model, train_data)
        AUC_test, MCC_test, (tn_test, fp_test, fn_test, tp_test) = metrics(trained_model, test_data)

        #результаты обучения (loss,auc за EPOCHS_COUNT эпох) сохраняются в словаре TRAIN_RESULTS_BY_SESSION
        TRAIN_RESULTS_BY_SESSION[s] = [AUC_train, MCC_train, (tn_train, fp_train, fn_train, tp_train)]
        TEST_RESULTS_BY_SESSION[s] = [AUC_test, MCC_test, (tn_test, fp_test, fn_test, tp_test)]

    return TRAIN_RESULTS_BY_SESSION, TEST_RESULTS_BY_SESSION, LOSSES

In [ ]:
TRAIN_RESULTS_BY_SESSION, TEST_RESULTS_BY_SESSION, LOSSES = cross_val_score(data)

SESSION № 05
Epoch 001: | Loss: 0.00000 | Acc: 25.036
Epoch 002: | Loss: 0.00000 | Acc: 25.070
Epoch 003: | Loss: 0.00000 | Acc: 25.074
Epoch 004: | Loss: 0.00000 | Acc: 25.034
Epoch 005: | Loss: 0.00000 | Acc: 25.010
Epoch 006: | Loss: 0.00000 | Acc: 25.006
Epoch 007: | Loss: 0.00000 | Acc: 25.080
tensor(0.2657) tensor(0.0597) tensor(-0.3113)
tensor(0.2909) tensor(0.0699) tensor(-0.2664)
SESSION № 06
Epoch 001: | Loss: 0.00000 | Acc: 69.567
Epoch 002: | Loss: 0.00000 | Acc: 69.477
Epoch 003: | Loss: 0.00000 | Acc: 69.462
Epoch 004: | Loss: 0.00000 | Acc: 69.523
Epoch 005: | Loss: 0.00000 | Acc: 69.632
Epoch 006: | Loss: 0.00000 | Acc: 69.500
Epoch 007: | Loss: 0.00000 | Acc: 69.446
tensor(0.5617) tensor(0.0892) tensor(0.0880)
tensor(0.5481) tensor(0.0873) tensor(0.0635)
SESSION № 07
Epoch 001: | Loss: 0.00000 | Acc: 76.770
Epoch 002: | Loss: 0.00000 | Acc: 76.716
Epoch 003: | Loss: 0.00000 | Acc: 76.763
Epoch 004: | Loss: 0.00000 | Acc: 76.703
Epoch 005: | Loss: 0.00000 | Acc: 76.757
